# ResNet

由于Neural ODE是受到了残差网络的启发的，所以有必要补充一下残差网络 ResNet 的基本概念。

本文主要参考了：

- [Introduction to ResNets](https://towardsdatascience.com/introduction-to-resnets-c0a830a288a4)
- [An Overview of ResNet and its Variants](https://towardsdatascience.com/an-overview-of-resnet-and-its-variants-5281e2f56035)
- [FrancescoSaverioZuppichini/ResNet](https://github.com/FrancescoSaverioZuppichini/ResNet)

残差网络的原论文是这篇：https://arxiv.org/pdf/1512.03385.pdf

2012 年，Krizhevsky 等人拉开了卷积神经网络的应用序幕。这是该架构第一次在 ImageNet 上比传统的、手工的特征学习更成功。他们的 DCNN 名为 AlexNet，包含 8 个神经网络层，5 个卷积层和 3 个全连接层。这为传统的 CNN 奠定了基础，卷积层后跟激活函数，然后是最大池化操作（有时会省略池化操作以保持图像的空间分辨率）。

深度神经网络的大部分成功都归功于这些附加层。其功能背后的直觉是这些层逐渐学习更复杂的特征。第一层学习边缘，第二层学习形状，第三层学习物体，第四层学习眼睛整体，依此类推。尽管AI 社区流行话语是“我们需要更深层”，但 He 等人经验表明，传统的 CNN 模型存在深度的最大阈值。

他们绘制了 20 层 CNN 与 56 层 CNN 的训练和测试误差。如下图所示。

![](imgs/0_fRYbrOU_YhS6oMf-.png)

这个图违背了我们的信念，即添加更多层会创建更复杂的函数，因此失败将归因于过度拟合。如果是这种情况，额外的正则化参数和算法（例如 dropout 或 L2-norms）将是修复这些网络的成功方法。然而，该图显示 56 层网络的训练误差也高于 20 层网络，突出了解释其失败的其他不同原因。

证据表明，使用卷积层和全连接层的最佳 ImageNet 模型通常包含 16 到 30 层。
56 层 CNN 的失败可能归咎于优化函数、网络初始化或著名的梯度消失/爆炸问题。梯度消失是尤其容易被归咎的一点，然而，作者认为批量归一化的使用确保梯度具有健康的规范。在解释深层网络为何无法比浅层网络表现更好的众多理论中，有时最好寻找实证结果进行解释，然后从那里倒推。通过引入新的神经网络层——残差块，训练非常深的网络的问题得到了缓解。

![](imgs/0_sGlmENAXIZhSqyFZ.png)

上图是这篇文章最值得学习的地方。对于希望快速实现并测试它的开发人员来说，要理解的最重要的修改是“跳过连接”(Skip Connections)，直接恒等映射（identity mapping）过去。这个identity mapping没有任何参数，只是将前一层的输出添加到前面的层。但是，有时 x 和 F(x) 不会具有相同的维度。回想一下，卷积操作通常会缩小图像的空间分辨率，例如 32 x 32 图像上的 3x3 卷积会产生 30 x 30 图像。所以用恒等映射乘以线性投影 W 的方式来扩展它以匹配残差。这允许将输入 x 和 F(x) 组合为下一层的输入。（当 F(x) 和 x 具有不同维度（例如 32x32 和 30x30）时使用的方程是：$y=F(x,{W_i})+W_sx$。这个 Ws 项可以用 1x1 卷积来实现，这会为模型引入额外的参数）

层之间的Skip Connections将前一层的输出添加到多层后面的输出。这使得训练比以前更深的网络成为可能。ResNet 架构的作者在 CIFAR-10 数据集上用 100 和 1,000 层测试他们的网络。他们在具有 152 层的 ImageNet 数据集上进行测试，该数据集的参数仍然少于 另一种非常流行的深度 CNN 架构 VGG 网络。一组深度残差网络在 ImageNet 上实现了 3.57% 的错误率，在 ILSVRC 2015 分类竞赛中获得第一名。

与 ResNets 类似的方法被称为“高速公路网络”(highway networks)。这些网络也实现了跳跃连接，然而，类似于 LSTM，这些跳跃连接通过参数门传递。这些门决定了有多少信息通过跳过连接。作者指出，当门接近关闭时，层代表非残差函数，而 ResNet 的恒等函数从未关闭。从经验上讲，作者指出，高速公路网络的作者并没有像使用 ResNets 所展示的那样深度展示网络的准确性提升。

总而言之，Skip Connection 是深度卷积网络的一个非常有趣的扩展，经验表明它可以提高 ImageNet 分类的性能。这些层也可用于需要深度网络的其他任务，例如定位、语义分割、生成对抗网络、超分辨率等。残差网络与 LSTM 不同，LSTM 对先前的信息进行门控，因此并非所有信息都通过。此外，本文中展示的 Skip Connections 基本上排列在 2 层块中，它们不使用相同的第 3 层到第 8 层的输入。残差网络更类似于注意力机制，因为它们对网络的内部状态进行建模反对输入。

下面是一个 残差块得实现。

In [9]:
import torch
from functools import partial

In [2]:
from torch import nn

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.in_channels, self.out_channels =  in_channels, out_channels
        self.blocks = nn.Identity()
        self.shortcut = nn.Identity()   
    
    def forward(self, x):
        residual = x
        if self.should_apply_shortcut: residual = self.shortcut(x)
        x = self.blocks(x)
        x += residual
        return x
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.out_channels

In [3]:
ResidualBlock(32, 64)

ResidualBlock(
  (blocks): Identity()
  (shortcut): Identity()
)

In [6]:
dummy = torch.ones((1, 1, 1, 1))

block = ResidualBlock(1, 64)
block(dummy)

tensor([[[[2.]]]])

上面得shortcut函数是简单定义的，直接和x相等，实际中可能是这样：

![](imgs/Block.png)

In [10]:
class Conv2dAuto(nn.Conv2d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.padding =  (self.kernel_size[0] // 2, self.kernel_size[1] // 2) # dynamic add padding based on the kernel_size
        
conv3x3 = partial(Conv2dAuto, kernel_size=3, bias=False)

In [11]:
from collections import OrderedDict

class ResNetResidualBlock(ResidualBlock):
    def __init__(self, in_channels, out_channels, expansion=1, downsampling=1, conv=conv3x3, *args, **kwargs):
        super().__init__(in_channels, out_channels)
        self.expansion, self.downsampling, self.conv = expansion, downsampling, conv
        self.shortcut = nn.Sequential(OrderedDict(
        {
            'conv' : nn.Conv2d(self.in_channels, self.expanded_channels, kernel_size=1,
                      stride=self.downsampling, bias=False),
            'bn' : nn.BatchNorm2d(self.expanded_channels)
            
        })) if self.should_apply_shortcut else None
        
        
    @property
    def expanded_channels(self):
        return self.out_channels * self.expansion
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.expanded_channels

In [12]:
ResNetResidualBlock(32, 64)

ResNetResidualBlock(
  (blocks): Identity()
  (shortcut): Sequential(
    (conv): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)